In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sujaymann/car-number-plate-dataset-yolo-format")

print("Path to dataset files:", path)

/opt/anaconda3/envs/VC_P4/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 203M/203M [00:12<00:00, 17.7MB/s] 

Extracting files...


Path to dataset files: /Users/nicolasreyalonso/.cache/kagglehub/datasets/sujaymann/car-number-plate-dataset-yolo-format/versions/3


SyntaxError: invalid syntax (1441000695.py, line 2)